# PDF Support

This notebook demonstrates PDF processing support for both `Call` and `Chat` objects. PDFs can be provided as URLs or local file paths with various parsing engines.

In [1]:
from irouter import Call, Chat
from irouter.base import nb_markdown

# To load OPENROUTER_API_KEY from .env file create a .env file at the root of the project with OPENROUTER_API_KEY=your_api_key
# Alternatively pass api_key=your_api_key to the Call or Chat class
from dotenv import load_dotenv

load_dotenv()

True

We'll use the free tier model for PDF processing demonstrations.

In [2]:
model = "moonshotai/kimi-k2:free"
pdf_url = "https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf"

# PDF URL



In [3]:
c = Call(model)

In [4]:
nb_markdown(c([pdf_url, "What is the main contribution of this paper?"]))

The main contribution of the paper is the introduction of the **Transformer**, a novel neural network architecture for sequence transduction tasks **based entirely on attention mechanisms**, replacing the commonly used recurrent or convolutional layers. This design enables:

1. **Superior performance** compared to prior state-of-the-art models (e.g., 28.4 BLEU on WMT 2014 English-to-German and 41.0 BLEU on English-to-French), **even outperforming ensembles**.
2. **Greater parallelizability**, allowing **faster training** (e.g., 12 hours on 8 GPUs for the base model).
3. **Elimination of sequential recurrence**, addressing the bottleneck of sequential computation in RNNs and enabling more efficient processing.

The paper also introduces key innovations like **multi-head self-attention**, **scaled dot-product attention**, and **positional encodings** to integrate sequence order without recurrence.

# PDF Parsing configuration

You can specify different PDF parsing engines using the `extra_body` parameter. For example, use `pdf-text` for free parsing. Check [this docs page](https://openrouter.ai/docs/features/multimodal/pdfs#plugin-configuration) for more details on plugin configuration.

In [5]:
extra_body = {"plugins": [{"id": "file-parser", "pdf": {"engine": "pdf-text"}}]}

In [6]:
nb_markdown(c([pdf_url, "Summarize the key innovations in this paper."], extra_body=extra_body))

The paper “Attention Is All You Need” presents **Transformer**, the first fully attention-based sequence-to-sequence model, eliminating recurrence and convolution entirely. Its key innovations are:

1. **Multi-Head Self-Attention**  
   - Replaces RNN/CNN layers with multiple parallel attention “heads” that jointly attend to information from different representation subspaces, enabling richer modeling and constant-time path lengths between any two positions.

2. **Scaled Dot-Product Attention**  
   - Introduces a simple, fast attention mechanism (matmul + scale + softmax) that avoids vanishing gradients by scaling dot-products by 1/√dₖ.

3. **Pure Attention Architecture**  
   - Encoder: 6 identical layers, each with (a) multi-head self-attention and (b) position-wise feed-forward network.  
   - Decoder: same 6 layers plus (c) encoder-decoder attention, plus autoregressive masking to prevent future-token leakage.

4. **Positional Encoding**  
   - Adds fixed sinusoidal position embeddings to word embeddings, letting the model exploit token order without recurrence or convolution and generalize to unseen sequence lengths.

5. **Massive Parallelization & Fast Training**  
   - Removing recurrence allows full sequence parallelization inside each example. Training a **big** Transformer on 8 P100 GPUs for 3.5 days beats the prior best single-model BLEU scores by **+2.0 (EN→DE)** and achieves **41.0 (EN→FR)**—both with markedly lower FLOPs than previous models.

6. **Residual/Layer-Norm Stacking**  
   - Each sub-layer uses residual connections and layer normalization, stabilizing deep stacks without recurrence.

7. **Shared Embedding Matrix**  
   - Ties input/output embedding weights and the pre-softmax projection matrix, reducing parameters.

Together, these advances yield the first sequence-to-sequence architecture where **attention is the only mechanism**, surpassing state-of-the-art results while training faster and scaling better.

# Chat with PDF

In [7]:
chat = Chat(model)

In [8]:
chat([pdf_url, "What is this paper about?"])

'This paper is about a new deep-learning architecture called the **Transformer**, which revolutionizes sequence-processing tasks (like machine translation) by **replacing recurrent or convolutional layers entirely with an attention mechanism**.\n\n### Key Contributions and Concepts:\n1. **Problem Addressed**:  \n   Traditional models (RNNs, CNNs) process sequences **sequentially**, limiting parallelization and long-range dependency modeling. Convolutions also struggle with distant relationships in sequences.\n\n2. **Solution**:  \n   The **Transformer** uses **only attention mechanisms (self-attention and multi-head attention)** to capture relationships between all positions in a sequence **in parallel**, making it faster to train and better at handling long-range dependencies.\n\n3. **Core Innovations**:\n   - **Scaled Dot-Product Attention**: A fast, parallelizable attention mechanism.\n   - **Multi-Head Attention**: Learns multiple types of relationships simultaneously.\n   - **Posi

Now we can ask follow-up questions about the PDF, and the chat will maintain context.

In [9]:
chat("What are the key advantages of this approach over RNNs?")

'Key advantages of the Transformer/self-attention over RNNs:\n\n1. **Parallelization**:  \n   RNNs process tokens step-by-step and must wait for the hidden state \u2006\\( h_{t-1} \\)\u2006 before computing \\( h_t \\), making sequential execution unavoidable. A Transformer layer computes attention over all positions simultaneously; the minimum **number of sequential operations is O(1) instead of O(n)**.\n\n2. **Shorter long-range paths**:  \n   A signal from position \\( i \\) to position \\( j \\) in an RNN has to traverse \\( O(n) \\) steps; self-attention creates a direct edge of constant length, so **maximum path length is O(1)** rather than O(n), making long-distance dependencies easier to learn.\n\n3. **Computation when \\( n < d \\)**:  \n   For typical translation sequences represented with word-piece/byte-pair tokens, \\( n \\) (sequence length) is much smaller than \\( d \\) (hidden dimension), so the self-attention layer’s total complexity \\( O(n^2 d) \\) can actually be *

In [10]:
chat.history

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': [{'type': 'file',
    'file': {'filename': 'document.pdf',
     'file_data': 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'}},
   {'type': 'text', 'text': 'What is this paper about?'}]},
 {'role': 'assistant',
  'content': 'This paper is about a new deep-learning architecture called the **Transformer**, which revolutionizes sequence-processing tasks (like machine translation) by **replacing recurrent or convolutional layers entirely with an attention mechanism**.\n\n### Key Contributions and Concepts:\n1. **Problem Addressed**:  \n   Traditional models (RNNs, CNNs) process sequences **sequentially**, limiting parallelization and long-range dependency modeling. Convolutions also struggle with distant relationships in sequences.\n\n2. **Solution**:  \n   The **Transformer** uses **only attention mechanisms (self-attention and multi-head a